In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini basic operation

## Set deveopement environment

### Package installation
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [18]:
!pip install --upgrade --quiet google-cloud-aiplatform

### Authentication to access to GCP
* Only Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.

In [2]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define contants

In [4]:
# Set the environment on GCP Project
PROJECT_ID="ai-hangsik"
REGION="us-central1"
MODEL_NAME="gemini-1.5-flash"

### Import libraries

In [6]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Part,
)

### Initialize Vertex AI

In [5]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=REGION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Helper function
* def generate(query:str, stream:bool)
  * Function to interact with Gemini to generate answer corresponding to query.

In [7]:
def generate(query:str, stream:bool) -> GenerationResponse:
    """
    Generate a response from the model.
    query :query to be sent to the model
    Returns:The generated response.
    """
    # Request body : https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#request
    # parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = GenerationConfig(
        temperature=0.9,
        top_p=1.0,
        top_k=32,
        candidate_count=1,
        max_output_tokens=8192,
    )

    contents = [query]

    # Counts tokens
    print(f"Token Count : {model.count_tokens(contents)}")

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    }
    responses = model.generate_content(
        contents,
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=stream,
    )
    return responses

In [17]:
from time import perf_counter

stream = True

t1_start = perf_counter()

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        다음 지침에서 AI를 사용하는 것의 장단점을 설명하세요.

        1. 생성 AI가 무엇인지 정의하세요.
        2. 일반적인 머신 러닝과 생성 AI의 차이점은 무엇인가요?
        3. 생성 AI를 사용하는 것의 장단점은 무엇인가요?
        4. 주요 산업에서 생성 AI의 사용 사례를 설명하세요.
"""

responses = generate(prompt, stream)

print(responses)

if stream:
  for response in responses:
    print(response.text, end=" ")

else:

  # Print the model response
  print(f'\n Usage metadata:\n{responses.to_dict().get("usage_metadata")}')
  print(f"\n Finish reason:\n{responses.candidates[0].finish_reason}")
  print(f"\n Safety settings:\n{responses.candidates[0].safety_ratings}")
  display(Markdown(responses.text))

t1_end  = perf_counter()

print(f"Time : {t1_end - t1_start} seconds\n\n")

Token Count : total_tokens: 125
total_billable_characters: 145

<generator object _GenerativeModel._generate_content_streaming at 0x7ca9a419b370>
생 성 AI 전문가로서 AI의 개념을 명확히 이해 하도록 돕겠습니다.

## 생성 AI란 무엇일까요? 

생성 AI는 새로운 콘텐츠를 생성하는 데 사용되는 인공 지능의 한 유형입니다. 텍스트, 이미지, 오디오,  비디오, 코드 등 다양한 형태의 콘텐츠를 생성할 수 있습니다. 이는 방대한 데이터 세트를 학습하여 생성 AI 모델이  새로운 콘텐츠를 생성하는 방법을 배우기 때문입니다.

## 생성 AI와 일반 머신 러닝의 차이점은 무엇인가요?

일반 머신 러닝은 주어진 데이터를 기반으로  예측을 수행하는 데 사용됩니다. 예를 들어, 이메일 스팸 여부를 분류하거나 주택 가격을 예측하는 등의 작업을 수행할 수 있습니다. 반면 생성 AI 는 새로운 콘텐츠를 생성하는 데 사용됩니다. 예를 들어, 새로운 음악을 작곡하거나 실제와 흡사한 이미지를 생성할 수 있습니다.

간략히 말해서, **일반 머신 러닝은 데이터를 분석하고 예측하는 데 사용되고, 생성 AI는  새로운 콘텐츠를 생성하는 데 사용**됩니다.

## 생성 AI를 사용하는 것의 장단점은 무엇일까요?

**장점:**

* **창의성과 생산성 향상:** 생성 AI는 새로운 아이디어를 생성하고 작업을 자동화하여 창의성 과 생산성을 높일 수 있습니다.
* **개인화된 콘텐츠 제공:** 생성 AI는 개인의 선호도에 맞는 콘텐츠를 제공하여 사용자 경험을 향상시킬 수 있습니다.
* **비용 절감:** 수동 작업을 자동화하여 비용을  절감하고 효율성을 높일 수 있습니다.
* **새로운 형태의 콘텐츠 생성:** 생성 AI는 기존의 방법으로는 생성할 수 없는 새로운 형태의 콘텐츠를 생성할 수 있습니다.

**단점:**

* **편견과 오류:** 생성 AI 모델 은 훈련 데이터에